Import dependencies

In [1]:
from model.layers import GraphSpectralFilterLayer, AnalysisFilter
from model.spectral_filter import Graph
import torch
import torch.nn.functional as F
from torch import nn
from random import seed as rseed
from numpy.random import seed as nseed
from citation import get_dataset, random_planetoid_splits, run
from citation.train_eval import evaluate
import numpy as np


Define hyperparameters

In [2]:
dataset_name = 'PubMed'

random_splits = False
runs = 1
alpha = 0.2
seed =729
weight_decay = 0.00012376256876336363
patience=10
hidden=88
lr =0.005
dropout=0.7
heads =12
output_heads =10
normalize_features = True
pre_training = False
cuda = False
order =16
edge_dropout =0
node_feature_dropout =0
filter_name ='analysis'

rseed(seed)
nseed(seed)
torch.manual_seed(seed)

cuda = cuda and torch.cuda.is_available()

if cuda:
    torch.cuda.manual_seed(seed)
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    
def get_correctly_predicted_node_idx(net, key, dataset):
    net.eval()
    with torch.no_grad():
        logits  = net(dataset[0])[0]
    mask = dataset[0]['{}_mask'.format(key)]
    pred = logits[mask].max(1)[1]
    return { *pred.eq(dataset[0].y[mask]).nonzero().view(-1).tolist() }

Define model

In [3]:
class Net(torch.nn.Module):
    def __init__(self, dataset):
        super(Net, self).__init__()
        data = dataset.data
        adj = torch.sparse_coo_tensor(data.edge_index, torch.ones(data.num_edges))
        self.G = Graph(adj)
        self.G.estimate_lmax()

        self.analysis = GraphSpectralFilterLayer(self.G, dataset.num_node_features, hidden,
                                                 dropout=dropout, out_channels=heads, filter=filter_name,
                                                 pre_training=False, device='cuda' if cuda else 'cpu',
                                                 alpha=alpha, order=order, concat=True)

        self.synthesis = GraphSpectralFilterLayer(self.G, hidden * heads, dataset.num_classes, filter=filter_name,
                                                  device='cuda' if cuda else 'cpu', dropout=dropout,
                                                  out_channels=1, alpha=alpha, pre_training=False,
                                                  order=order, concat=False)

    def reset_parameters(self):
        self.analysis.reset_parameters()
        self.synthesis.reset_parameters()

    def forward(self, data):
        x = data.x
        x = F.dropout(x, p=dropout, training=self.training)
        x, att1 = self.analysis(x)
        x = F.dropout(x, p=dropout, training=self.training)
        x, att2 = self.synthesis(x)
        last_layer = x
        x = F.elu(x)
        return F.log_softmax(x, dim=1), last_layer, None

dataset = get_dataset(dataset_name, normalize_features, edge_dropout=edge_dropout,
                                node_feature_dropout=node_feature_dropout)

if cuda:
    dataset[0].to('cuda')

In [4]:
student_heads = 1

class StudentNet(torch.nn.Module):
    def __init__(self, dataset):
        super(StudentNet, self).__init__()
        data = dataset.data
        adj = torch.sparse_coo_tensor(data.edge_index, torch.ones(data.num_edges))
        self.G = Graph(adj)
        self.G.estimate_lmax()

        self.analysis = GraphSpectralFilterLayer(self.G, dataset.num_node_features, dataset.num_classes,
                                                 dropout=dropout, out_channels=student_heads, filter=filter_name,
                                                 pre_training=False, device='cuda' if cuda else 'cpu',
                                                 alpha=alpha, order=order, concat=False)

        # self.synthesis = GraphSpectralFilterLayer(self.G, hidden * student_heads, dataset.num_classes, filter=filter_name,
        #                                           device='cuda' if cuda else 'cpu', dropout=dropout,
        #                                           out_channels=student_heads, alpha=alpha, pre_training=False,
        #                                           order=order, concat=False)
    def reset_parameters(self):
        self.analysis.reset_parameters()

    def forward(self, data):
        x = data.x
        x = F.dropout(x, p=dropout, training=self.training)
        x, att1 = self.analysis(x)
        # x = F.dropout(x, p=dropout, training=self.training)
        # last_layer, att2 = self.synthesis(x)
        last_layer = x
        x = F.elu(x)
        return F.log_softmax(x, dim=1), last_layer, None

Load trained model and evaluate

In [5]:
model = Net(dataset)
model.load_state_dict(torch.load('./model/best_{}_gpu.pkl'.format(dataset_name),  map_location={'cuda:0': 'cpu'}))

# model = SingleNet(dataset)
# model.load_state_dict(torch.load('./model/best_{}_single_layer.pkl'.format(dataset_name)))

# filter_kernel = model.analysis.filter_kernel

# model_correct_indices = get_correctly_predicted_node_idx(model, 'test', dataset)
eval_info = evaluate(model, dataset[0])
print(eval_info)

{'train_loss': 0.25815349817276, 'train_acc': 0.9092056869956634, 'train_micro_f1': 0.9092056869956634, 'train_macro_f1': 0.9060534433900731, 'val_loss': 0.2487691044807434, 'val_acc': 0.916, 'val_micro_f1': 0.916, 'val_macro_f1': 0.9116419796876049, 'test_loss': 0.27245962619781494, 'test_acc': 0.903, 'test_micro_f1': 0.903, 'test_macro_f1': 0.8965550190064704}


In [6]:
with torch.no_grad():
    soft_target = model(dataset[0])[1]

In [7]:
from torch.optim import Adam
from sklearn.metrics import f1_score
student = StudentNet(dataset)
student.reset_parameters()
optimizer = Adam(student.parameters(), lr=lr, weight_decay=weight_decay)
data = dataset.data

epochs =2000
lr =0.02
dropout=0
best_val_loss = float('inf')
best_val_acc = float(0)
eval_info_early_model = None
bad_counter = 0

for epoch in range(1, epochs + 1):
    optimizer.zero_grad()
    logits, out, _ = student(data)
    loss = F.mse_loss(out, soft_target)
    loss.backward()
    optimizer.step()
    student.train()

    # eval_info['epoch'] = epoch
    if epoch % 10 == 0:
        student.eval()
        outs = {}
        outs['loss'] = loss.item()
        for key in ['train', 'val', 'test']:
            mask = data['{}_mask'.format(key)]
            loss = F.nll_loss(logits[mask], data.y[mask]).item()
            pred = logits[mask].max(1)[1]

            outs['{}_loss'.format(key)] = loss

            outs['{}_micro_f1'.format(key)] = f1_score(data.y[mask].cpu(), logits[mask].max(1)[1].cpu(), average='micro')
            outs['{}_macro_f1'.format(key)] = f1_score(data.y[mask].cpu(), logits[mask].max(1)[1].cpu(), average='macro')

        print(outs)
    # if eval_info['val_acc'] > best_val_acc or eval_info['val_loss'] < best_val_loss:
    #     if eval_info['val_acc'] >= best_val_acc and eval_info['val_loss'] <= best_val_loss:
    #         eval_info_early_model = eval_info
    #         # torch.save(model.state_dict(), './best_{}_appnp.pkl'.format(dataset.name))
    #     best_val_acc = np.max((best_val_acc, eval_info['val_acc']))
    #     best_val_loss = np.min((best_val_loss, eval_info['val_loss']))
    #     bad_counter = 0
    # else:
    #     bad_counter += 1
    #     if bad_counter == patience:
    #         break

{'loss': 6.957119941711426, 'train_loss': 1.0753040313720703, 'train_micro_f1': 0.5222594280068068, 'train_macro_f1': 0.5004430200562994, 'val_loss': 1.07261061668396, 'val_micro_f1': 0.52, 'val_macro_f1': 0.49669192830506437, 'test_loss': 1.0748200416564941, 'test_micro_f1': 0.531, 'test_macro_f1': 0.501969511969512}
{'loss': 6.658788681030273, 'train_loss': 1.0564773082733154, 'train_micro_f1': 0.5399352253389691, 'train_macro_f1': 0.5084354776653283, 'val_loss': 1.0529581308364868, 'val_micro_f1': 0.538, 'val_macro_f1': 0.5117738873691966, 'test_loss': 1.0556169748306274, 'test_micro_f1': 0.55, 'test_macro_f1': 0.5171089205866775}
{'loss': 6.376893043518066, 'train_loss': 1.036821722984314, 'train_micro_f1': 0.5497063182741395, 'train_macro_f1': 0.5103197602212798, 'val_loss': 1.0325250625610352, 'val_micro_f1': 0.57, 'val_macro_f1': 0.5339270344070224, 'test_loss': 1.035496711730957, 'test_micro_f1': 0.556, 'test_macro_f1': 0.5187831344728614}
{'loss': 6.111839294433594, 'train_los